# Лабораторная работа №2
# Решение задач линейного программирования при помощи симплекс-метода
---
Выполнил: Совпель Дмитрий, гр. 853501

## Реализация метода

Импортируем необходимые библиотеки.

In [1]:
import typing
import warnings
import numpy as np

import scipy.linalg as sla
import scipy.optimize as opt

Метод нахождения первоначальной матрицы основан на решении системы линейных уравнений с использованием библиотеки `scipy` и ее функции `solve`. Метод обращения из предыдущей лабораторной работы не использовался, так как точность метода была ниже, чем у библиотечных реализаций. 
При необходимости можно было воспользоватся методами обращения на основе `PLU` или `QR` разложений матрицы, а также методами `LMGRES` или методом сопряженных градиентов (`CG`) при решении систем с разрежянными матрицами, однако в данной лабораторной работе рассматривается наиболее общая постановка задачи. 

In [2]:
def inverse(A: np.array) -> np.array:
    return sla.solve(A, np.eye(A.shape[0]))

Реализуем симплекс-метод. При этом, в данной реализации был сделан упор на максимальное использования библиотеки `numpy` для работы с векторами и матрицами, нахождения минимума в векторе, использовании предикатов относительно вектора и в целом полной минимазации использовании обычных циклов. Это было сделано по ряду причин:
- использование обычных циклов Python очень сильно замедляет программу, `numpy` использует код, написанный на языке `C` (тем самым более быстрый), и оптимизированный для векторно-матричных операций
- реализация более универсальна с точки зрения представления входных данных - при необходимости работы с разряженными матрицами программу можно легко изменить для данных целей  

In [3]:
def simplex(A: np.array, c: np.array, initial_basic_solution: np.array, initial_basis: list, verbose: bool = False) -> np.array:
    iteration_no = 0

    basis = initial_basis.copy()
    non_basis = list(set(range(initial_basic_solution.size)) - set(basis))
    
    solution = initial_basic_solution.copy()
    basic_A_inverse = inverse(A[:, basis])
    if verbose:
        print(f'Initial B:\n {basic_A_inverse}')
        
    while True:
        potential = c[basis] @ basic_A_inverse
        estimate = potential @ A[:, non_basis] - c[non_basis]
        estimate_negative_or_first_element_index = np.argmax(estimate < 0.0)    
        j_0 = non_basis[estimate_negative_or_first_element_index]
        if verbose:
            print(f'Iteration no.: {iteration_no}')
            print(f'Vector potential: {potential}')
            print(f'Vector estimate: {estimate}')
            
        if estimate[estimate_negative_or_first_element_index] >= 0.0:
            return solution, basis
        else:
            z = basic_A_inverse @ A[:, j_0] 
            if verbose:
                print(f'Index j_0: {j_0 + 1}')
                print(f'Vector z: {z}')
            
            if np.any(z > 0):
                positive_z_indexes = np.where(z > 0)[0]
                solution_over_z = solution[basis][positive_z_indexes] / z[positive_z_indexes]
                solution_over_z_argmin = np.argmin(solution_over_z)
                
                s = positive_z_indexes[solution_over_z_argmin]
                j_s = basis[s]
                theta = solution_over_z[solution_over_z_argmin]                
                if verbose:
                    print(f'Index s: {s + 1}')
                    print(f'Index j_s: {j_s + 1}')
                    print(f'Theta: {theta}')
                
                solution[basis] = solution[basis] - theta * z
                solution[j_0] = theta
                
                basis[basis.index(j_s)] = j_0
                non_basis[non_basis.index(j_0)] = j_s
                
                z_s = z[s]
                z[s] = -1
                z /= -z_s 

                M = np.eye(z.size)
                M[:, s] = z
                basic_A_inverse = M @ basic_A_inverse
                if verbose:
                    print(f'Solution: {solution}')
                    print(f'Basis: {[i + 1 for i in basis]}')
                    print(f'B:\n {basic_A_inverse}')
            else:
                warnings.warn('No solution', opt.OptimizeWarning)
                return None, None
        iteration_no += 1

## Постановка линейной программы на основе реальной задачи и тестирование  симплекс-метода на ней

Рассмотрим групповую задачу 1.20 - предприятие Танти Мару. Для того, чтобы составить линейную программу, необходимо определится с тем, что для данной практической задачи является планом, затем на основании условия составить ограничения на этот план и оптимизируемый функционал. 

В качестве плана я выбрал вектор $x \in R^{15}$, где каждый элемент вектора $x_i$ представляет собой количество (не обязательно целое, в рамках данной формализации линия может быть не полностью нагруженной) запущенных  линиий определенного типа (первого или второго, или третьего) в определенный день недели (с понедельника по пятницу). 

Составим прямые ограничения для плана $x$: 
- $0 \le x$ - (запущенных линий не может быть отрицательное количество)
- $x \le d$, где $d = \begin{pmatrix} 8 & 8 & 3 & 8 & 8 & 3 & 8 & 8 & 3 & 8 & 8 & 3 & 8 & 8 & 3\end{pmatrix}^T$ - из ограничений на количество линий каждого типа

Составим основные ограничения для плана $x$:
- $Ax \ge b$, где $A = \begin{pmatrix} 
150 & 200 & 400 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
150 & 200 & 400 & 150 & 200 & 400 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
150 & 200 & 400 & 150 & 200 & 400 & 150 & 200 & 400 & 0 & 0 & 0 & 0 & 0 & 0 \\
150 & 200 & 400 & 150 & 200 & 400 & 150 & 200 & 400 & 150 & 200 & 400 & 0 & 0 & 0 \\
150 & 200 & 400 & 150 & 200 & 400 & 150 & 200 & 400 & 150 & 200 & 400 & 150 & 200 & 400 \\
\end{pmatrix}$, а $b = \begin{pmatrix} 3000 \\ 6800 \\ 11000 \\ 14650 \\ 17150 \end{pmatrix}$. Данные ограничения возникают из условий, связанных с производительностью каждой линии, а также возможностью производить продукт с запасом.


Составим оптимизируемый функционал: $c^Tx \rightarrow min$, где $c = \begin{pmatrix} 250 + 30 * 150 \\ 500 + 20 * 200 \\ 700 + 18 * 400 \\ 250 + 30 * 150 \\ 500 + 20 * 200 \\ 700 + 18 * 400 \\ 250 + 30 * 150 \\ 500 + 20 * 200 \\ 700 + 18 * 400 \\ 250 + 30 * 150 \\ 500 + 20 * 200 \\ 700 + 18 * 400 \\ 250 + 30 * 150 \\ 500 + 20 * 200 \\ 700 + 18 * 400 \end{pmatrix} = \begin{pmatrix} 4750 \\ 4500 \\ 7900 \\ 4750 \\ 4500 \\ 7900 \\ 4750 \\ 4500 \\ 7900 \\ 4750 \\ 4500 \\ 7900 \\ 4750 \\ 4500 \\ 7900\end{pmatrix}$

Теперь, для применения реализованного симплекс-метода необходимо привести линейную программу вида:

$$ \begin{cases}
c^Tx \rightarrow min \\
Ax \ge b \\
0 \le x \le d
\end{cases}
$$
в каноническую форму:
$$
    \begin{cases}
      c^Tx \rightarrow max \\
      Ax = b \\
      x \ge 0 \\
     \end{cases}
$$

Преобразуем прямые ограничения используя slack variable $y$:
$$z + y = d, z \ge 0, y \ge 0$$

Тем самым получили линейную программу вида:
$$
    \begin{cases}
        c^Tz \rightarrow min \\
        Az = b \\
        z + y = d \\
        z \ge 0 \\
        y \ge 0
    \end{cases}
$$

Преобразуем основные ограничения используя slack variable $t$:
$$Az - Et = b$$

Получим линейную программу вида:
$$
    \begin{cases}
        c^Tz \rightarrow min \\
        Az - Et = b \\
        z + y = d \\
        z \ge 0 \\
        y \ge 0 \\
        t \ge 0
    \end{cases}
$$

Такжe, преобразуем минимизируемый функционал (чтобы получить задачу максимизации):
$$ \overline{c} = -c$$

Окончательно, получим линейную программу вида:
$$
    \begin{cases}
        \overline{c^T}z \rightarrow max \\
        Az - Et = b \\
        z + y = d \\
        z \ge 0 \\
        y \ge 0 \\
        t \ge 0
    \end{cases}
$$

Чтобы привести данную линейную программу к каноническому виду используем блочные векторы и матрицы:
$$
\begin{aligned}
\overline{X} &= (z, t, y) \\
\overline{A} &= \begin{pmatrix} A & -E & 0 \\ E & 0 & E\end{pmatrix} \\
\overline{B} &= \begin{pmatrix} b \\ d \end{pmatrix} \\
\overline{C} &= \begin{pmatrix} \overline{c} \\ 0 \\ 0\end{pmatrix}
\end{aligned}$$

Таким образом получаем линенейную программу, заданную в канонической форме:
$$\begin{cases}
\overline{C} \space \overline{X} \rightarrow \max \\
\overline{A}\space \overline{X} = \overline{B} \\
\overline{X} \ge 0
\end{cases}$$

Перепишем начальные условия в виде `numpy` векторов и матриц

In [4]:
A = np.array([[150, 200, 400, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[150, 200, 400, 150, 200, 400, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[150, 200, 400, 150, 200, 400, 150, 200, 400, 0, 0, 0, 0, 0, 0],
[150, 200, 400, 150, 200, 400, 150, 200, 400, 150, 200, 400, 0, 0, 0],
[150, 200, 400, 150, 200, 400, 150, 200, 400, 150, 200, 400, 150, 200, 400]], dtype='float32')

b = np.array([3000, 6800, 11000, 14650, 17150], dtype='float32')
c = np.array([4750, 4500, 7900, 4750, 4500, 7900, 4750, 4500, 7900, 4750, 4500, 7900, 4750, 4500, 7900], dtype='float32')
d = np.array([8, 8, 3, 8, 8, 3, 8, 8, 3, 8, 8, 3, 8, 8, 3], dtype='float32')

Составим блочные матрицы $\overline{A}, \overline{B}, \overline{C}$, а также запишем базовый план $X$ и по нему составим базис.  

In [5]:
overline_A = np.block([[A, -np.eye(b.size), np.zeros((b.size, d.size))], [np.eye(d.size), np.zeros((d.size, b.size)), np.eye(d.size)]])
overline_C = np.block([-c, np.zeros(b.size), np.zeros(d.size)])
overline_B = np.block([b, d])
initial_solution = np.array([1.33, 8.00, 3.00, 8.00, 8.00, 3.00, 8.00, 8.00, 3.00, 5.67,
                             8.00, 3.00, 0, 6.50, 3.00, 0, 2.00, 0.05, 0, 0, 6.66, 0, 0, 0, 0, 0, 0, 0, 0, 2.33, 0, 0, 8.00, 1.50, 0])
initial_basis = [index for index, element in enumerate(initial_solution) if not np.isclose(element, 0.0)]

Решим данную линейную программу при помощи разработанного симплекс метода. Затем, получим план изначальной линейной программы $x$, выбрав только первые 15 значений полученного плана (то есть, избавимся от slack variables)

In [6]:
solution, solution_basis = simplex(A=overline_A, c=overline_C, initial_basic_solution=initial_solution, initial_basis=initial_basis)
final_solution = solution[:15]

Таким образом, мы получили оптимальный план $x$ исходной задачи 

In [7]:
final_solution

array([7.99, 8.  , 3.  , 7.01, 8.  , 3.  , 8.  , 8.  , 3.  , 0.  , 8.  ,
       3.  , 0.  , 6.5 , 3.  ])

При этом, минимум оптимизируемого функционала равен

In [8]:
c @ final_solution

401000.0

## Тестирование реализованного алгоритма на линейной программе согласно варианту для определения соответствия алгоритма заложенному симплекс-методу

Рассмотрим ход работы симплекс-метода для линейной программы (заданной в канонической форме) согласно варианту 25:
$$
    \begin{cases}
      c^Tx \rightarrow max \\
      Ax = b \\
      x > 0 \\
     \end{cases}
$$
где: 
$$
    \begin{aligned} 
        A &= \begin{pmatrix} 1 & -1 & 0 & 2 & -1 & -1 & 2 & 1 & 7 \\
    -1 & 2 & 1 & 3 & 0 & 1 & 1 & 1 & 1 \\
    1 & -1 & 1 & 0 & 1 & 4 & 4 & 1 & 2 \\
    1 & -3 & 0 & -8 & 2 & 3 & 0 & 1 & 1 \\
    \end{pmatrix} \\ \\
    b &= \begin{pmatrix} -2 & 10 & 25 & 14\end{pmatrix} \\ \\
    c &= \begin{pmatrix} 4 & -5 & 1 & 3 & 1 & 8 & 1 & 1 & -1  \end{pmatrix} 
    \end{aligned}
$$
при этом необходимо взять базисный план $x$:
$$
    x = \begin{pmatrix} 0 & 1 & 0 & 0 & 0 & 5 & 1 & 2 & 0\end{pmatrix}
$$
и базис:
$$
    J = \{2, 6, 7, 8\}
$$

**Итерация** $i = 0$

По заданному множеству базисному множеству $J_B = \{2, 6, 7, 8\}$ сформируем базисную матрицу $A_B$:

$$A_B = \begin{pmatrix} 
-1 & -1 & 2 & 1 \\ 
2 & 1 & 1 & 1 \\
-1 & 4 & 4 & 1 \\
-3 & 3 & 0 & 1 \\
\end{pmatrix}$$

Вычислим обратную матрицу $B = A_B^{-1}$: 

$$ B = 
\begin{pmatrix} 
-0.125 & 0.25 & 0 & -0.125 \\
-0.25 & 0.055 & 0.11 & 0.083 \\
0.125 & -0.138 & 0.22 & -0.208 \\
0.375 & 0.583 & -0.333 & 0.375
\end{pmatrix}$$

Построим $c_B = \begin{pmatrix} -5 \\ 8 \\ 1 \\ 1\end{pmatrix}$.

Вычислим вектор потенциалов $u^T$ и вектор оценок $\Delta$: 

$$ u^T = c^T_BB = \begin{pmatrix} -5 & 8 & 1 & 1\end{pmatrix} \begin{pmatrix} -0.125 & 0.25 & 0 & -0.125 \\
-0.25 & 0.055 & 0.11 & 0.083 \\
0.125 & -0.138 & 0.22 & -0.208 \\
0.375 & 0.583 & -0.333 & 0.375
\end{pmatrix} = \begin{pmatrix}-0.875 & -0.361 & 0.778 & 1.458 \end{pmatrix} 
\\
\\
\Delta = u^TA - c^T = \begin{pmatrix}-0.875 & -0.361 & 0.778 & 1.458 \end{pmatrix} \begin{pmatrix} 1 & -1 & 0 & 2 & -1 & -1 & 2 & 1 & 7 \\
-1 & 2 & 1 & 3 & 0 & 1 & 1 & 1 & 1 \\
1 & -1 & 1 & 0 & 1 & 4 & 4 & 1 & 2 \\
1 & -3 & 0 & -8 & 2 & 3 & 0 & 1 & 1 \\
\end{pmatrix} - \begin{pmatrix}4 & -5 & 1 & 3 & 1 & 8 & 1 & 1 & -1\end{pmatrix} = \begin{pmatrix}-2.278 & 0 & -0.583 & -17.5 & 3.570 & 0 & 0 & 0 & -2.472\end{pmatrix}$$

Для данного вектора оценок условие $\Delta_j \ge 0, j \in J_H = J \backslash J_B$ не соблюдается. Выберем индекс по правилу Бленда $j_0 \in J_H$, для которого $\Delta_{j_0} < 0$: 

$$\Delta_1 = -2.278 < 0 \Rightarrow j_0 = 1$$

Построим вектор $z$:

$$z = BA_{j_0} = \begin{pmatrix} -0.125 & 0.25 & 0 & -0.125 \\
-0.25 & 0.055 & 0.11 & 0.083 \\
0.125 & -0.138 & 0.22 & -0.208 \\
0.375 & 0.583 & -0.333 & 0.375
\end{pmatrix} \begin{pmatrix} 1 & -1 & 1 & 1\end{pmatrix} = \begin{pmatrix} -0.5 \\ -0.11111111 \\ 0.27777778 \\ -0.16666667\end{pmatrix}$$

Для данного вектора $z$ условие $z \le 0$, не выполняется, поэтому продолжим итерацию. Вычислим $\theta_i, i=\overline {1,4}$:

$$\theta_i = \begin{cases} 
\frac{x_{j_i}}{z_i}, z_i > 0 \\
+\infty, z_i \le 0
\end{cases}, \forall j \in J_B$$
Найдем $\theta_0 = \min{\theta_i} \space \forall i \in \overline{1, 4}$:

$$\theta_0 = \theta_3 = 3.6 \Rightarrow s = 3, j_s = j_3 = 7$$

Построим новые базис и базисный план $\overline{x}_j$ по формулам:

$$
\overline{x}_j = 0, j \in J_H \backslash j_0 \\ \overline{x}_{j_0} = \theta_0 \\ \overline{x}_{j_i} = x_{j_i} - \theta_0z_i i \in \overline{1, 4} \\
\overline{J}_B = (J_B \backslash j_s) \cup j_0
$$
Получим новые базис $\overline{J}_B = \{2, 6, 1, 8\}$ и план $x = \begin{pmatrix}3.6 & 2.8 & 0 & 0 & 0 & 5.4 & 0 & 2.6 & 0\end{pmatrix}$
Также, необходимо пересчитать обратную базисную матрицу $\overline{B}$. Из формул прошлой лабораторной работы получим:

$$\overline{B} = \begin{pmatrix}
0.1 & 0 & 0.4 & -0.5 \\
-0.2 & 0 & 0.2 & 0 \\
0.45 & -0.5 & 0.8 & -0.75 \\
0.45 & 0.5 & -0.2 & 0.25 
\end{pmatrix}$$

**Итерация** $i = 1$

По полученному из предыдущей итерации базисному множеству $J_B=\{2, 6, 1, 8\}$ сформируем базисную матрицу $A_B$:

$$A_B = \begin{pmatrix}
-1 & -1 & 1 & 1 \\ 
2 & 1 & -1 & 1 \\
-1 & 4 & 1 & 1 \\
-3 & 3 & 1 & 
\end{pmatrix}$$

Также используем полученную на предыдущем шаге обратную матрицу $B$:

$$B = \begin{pmatrix}
0.1 & 0 & 0.4 & -0.5 \\
-0.2 & 0 & 0.2 & 0 \\
0.45 & -0.5 & 0.8 & -0.75 \\
0.45 & 0.5 & -0.2 & 0.25 
\end{pmatrix}$$

Построим $c_B = \begin{pmatrix} -5 \\ 8 \\ 4 \\ 1 \end{pmatrix}$.

Вычислим вектор потенциалов $u^T$ и вектор оценок $\Delta$:

$$ u^T = c^T_BB =  \begin{pmatrix} -5 & 8 & 4 & 1 \end{pmatrix} \begin{pmatrix}
0.1 & 0 & 0.4 & -0.5 \\
-0.2 & 0 & 0.2 & 0 \\
0.45 & -0.5 & 0.8 & -0.75 \\
0.45 & 0.5 & -0.2 & 0.25 
\end{pmatrix} = \begin{pmatrix} 0.15 & -1.5 & 2.6 & -0.25 \end{pmatrix} \\
\Delta = u^TA - c^T = \begin{pmatrix} 0.15 & -1.5 & 2.6 & -0.25  \end{pmatrix} \begin{pmatrix} 1 & -1 & 0 & 2 & -1 & -1 & 2 & 1 & 7 \\
-1 & 2 & 1 & 3 & 0 & 1 & 1 & 1 & 1 \\
1 & -1 & 1 & 0 & 1 & 4 & 4 & 1 & 2 \\
1 & -3 & 0 & -8 & 2 & 3 & 0 & 1 & 1 \\
\end{pmatrix} - \begin{pmatrix}4 & -5 & 1 & 3 & 1 & 8 & 1 & 1 & -1\end{pmatrix} = \begin{pmatrix} 0 & 0 & 0.1 & -5.2 & 0.95 & 0 & 8.2 & 0 & 5.2 \end{pmatrix}$$

Для данного вектора оценок условие $\Delta_j \ge 0, j \in J_H = J \backslash J_B$ не соблюдается. Выберем индекс по правил Бленда $j_0 \in J_H$, для которого $\Delta_{j_0} < 0$:

$$\Delta_4 = -5.2 < 0 \Rightarrow j_0 = 4$$

Построим вектор $z$: 

$$z = BA_{j_0} =  \begin{pmatrix}
0.1 & 0 & 0.4 & -0.5 \\
-0.2 & 0 & 0.2 & 0 \\
0.45 & -0.5 & 0.8 & -0.75 \\
0.45 & 0.5 & -0.2 & 0.25 
\end{pmatrix} \begin{pmatrix} 2 \\ 3 \\ 0 \\ -8\end{pmatrix} = \begin{pmatrix} 4.2 & -0.4 & 5.4 & 0.4\end{pmatrix}$$

Для данного вектора $z$ условие $z \le 0$, не выполняется, поэтому продолжим итерацию. Вычислим $\theta_i, i=\overline {1,4}$:

$$\theta_i = \begin{cases} 
\frac{x_{j_i}}{z_i}, z_i > 0 \\
+\infty, z_i \le 0
\end{cases}, \forall j \in J_B$$

Найдем $\theta_0 = \min{\theta_i} \space \forall i \in \overline{1, 4}$:

$$\theta_0 = \theta_3 = 0.66 \Rightarrow s = 3, j_s = j_3 = 1$$

Построим новые базис и базисный план $\overline{x}_j$ по формулам:

$$
\overline{x}_j = 0, j \in J_H \backslash j_0 \\ 
\overline{x}_{j_0} = \theta_0 \\ 
\overline{x}_{j_i} = x_{j_i} - \theta_0z_i i \in \overline{1, 4} \\
\overline{J}_B = (J_B \backslash j_s) \cup j_0
$$
Получим новые базис $\overline{J}_B = \{2, 6, 4, 8\}$ и план $x = \begin{pmatrix}0 & 0 & 0 & 0.67 & 0 & 0.57 & 0 & 0.23 & 0 \end{pmatrix}$

Также, необходимо пересчитать обратную базисную матрицу $\overline{B}$. Из формул прошлой лабораторной работы получим:

$$\overline{B} = \begin{pmatrix}
-0.25 & 0.39 & -0.22 & 0.08 \\
-0.17 & -0.04 & 0.26 & -0.06 \\
0.08 & -0.09 & 0.15 & -0.14 \\
0.42 & 0.54 & -0.26 &  0.31
\end{pmatrix}$$

Дальнейшие итерации выполняются аналогично до момента, когда очередной вектор потенциалов $\Delta$ по всем небазисным индексам $j \in J \backslash J_B$ будет иметь неотрицательные элементы.

Запустив реализованный алгоритм можно увидеть, что выводимые промежуточные результаты совпадают (с точностью до перестановки элементов) с двумя разобранными шагами.

In [9]:
A = np.array([[1, -1, 0, 2, -1, -1, 2, 1, 7 ],
            [-1, 2, 1, 3, 0, 1, 1, 1, 1],
            [1, -1, 1, 0, 1, 4, 4, 1, 2],
            [1, -3, 0, -8, 2, 3, 0, 1, 1]], dtype='float64')
b = np.array([-2, 10, 25, 14], dtype='float64')
c = np.array([4, -5, 1, 3, 1, 8, 1, 1, -1], dtype='float64')
initial_solution = np.array([0, 1, 0, 0, 0, 5, 1, 2, 0], dtype='float64')
basis = [1, 5, 6, 7]

simplex(A, c, initial_solution, basis, True)

Initial B:
 [[-1.25000000e-01  2.50000000e-01  1.85037171e-17 -1.25000000e-01]
 [-2.50000000e-01  5.55555556e-02  1.11111111e-01  8.33333333e-02]
 [ 1.25000000e-01 -1.38888889e-01  2.22222222e-01 -2.08333333e-01]
 [ 3.75000000e-01  5.83333333e-01 -3.33333333e-01  3.75000000e-01]]
Iteration no.: 0
Vector potential: [-0.875      -0.36111111  0.77777778  1.45833333]
Vector estimate: [ -2.27777778  -0.58333333 -17.5          3.56944444  -2.47222222]
Index j_0: 1
Vector z: [-0.5        -0.11111111  0.27777778 -0.16666667]
Index s: 3
Index j_s: 7
Theta: 3.5999999999999996
Solution: [3.6 2.8 0.  0.  0.  5.4 0.  2.6 0. ]
Basis: [2, 6, 1, 8]
B:
 [[ 1.00000000e-01  3.88578059e-17  4.00000000e-01 -5.00000000e-01]
 [-2.00000000e-01 -3.53112601e-17  2.00000000e-01  4.62592927e-19]
 [ 4.50000000e-01 -5.00000000e-01  8.00000000e-01 -7.50000000e-01]
 [ 4.50000000e-01  5.00000000e-01 -2.00000000e-01  2.50000000e-01]]
Iteration no.: 1
Vector potential: [ 0.15 -1.5   2.6  -0.25]
Vector estimate: [ 8.2   

(array([0.00000000e+00, 4.93038066e-32, 0.00000000e+00, 6.66666667e-01,
        2.28388736e-15, 5.66666667e+00, 0.00000000e+00, 2.33333333e+00,
        0.00000000e+00]),
 [4, 5, 3, 7])

Также, можно проверить результат при помощи симплекс-метода реализованного в функции `linprog` из библиотеки `scipy`. Как видно, результаты совпадают. 

In [10]:
opt.linprog(A_eq=A, b_eq=b, c=-c, x0=initial_solution, method='revised simplex')

     con: array([-1.11022302e-15,  0.00000000e+00,  3.55271368e-15,  3.55271368e-15])
     fun: -49.666666666666664
 message: 'Optimization terminated successfully.'
     nit: 3
   slack: array([], dtype=float64)
  status: 0
 success: True
       x: array([2.85485921e-16, 1.11022302e-16, 0.00000000e+00, 6.66666667e-01,
       0.00000000e+00, 5.66666667e+00, 0.00000000e+00, 2.33333333e+00,
       0.00000000e+00])